In [1]:
SCRIPT_DIR = os.path.dirname('../src/')
sys.path.append(os.path.dirname(SCRIPT_DIR))

In [2]:
import warnings
import numpy as np
import pandas as pd 

from sklearn.preprocessing import LabelEncoder, MinMaxScaler

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from src.wrapped import Wrapped

In [25]:
warnings.filterwarnings("ignore")

wp = Wrapped(
    '../data/row/',
    '../data/processed/',
    '../data/files/'
)

# Importando dados para treinamento

In [26]:
df = wp.load_data('df_instrumentos')
df

,instrumento,file_name,chroma1,chroma2,chroma3,chroma4,chroma5,chroma6,chroma7,chroma8,...,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19
0,accordion,000139_119040.ogg,0.513887,0.581963,0.434641,0.293949,0.317826,0.271148,0.548258,0.660767,...,-5.140857,10.352439,-1.830431,0.709631,-1.533663,3.445959,1.397324,7.555676,1.468902,10.498053
1,accordion,000145_172800.ogg,0.175587,0.429345,0.840043,0.384524,0.282545,0.346341,0.482365,0.321750,...,-1.211487,9.595064,-0.532024,9.224909,-1.727268,0.398663,-5.348599,5.749533,-2.110756,-3.542143
2,accordion,000201_168960.ogg,0.453814,0.219808,0.228484,0.446050,0.178916,0.250981,0.327460,0.333150,...,-16.190577,1.937491,-9.938826,-7.672649,-12.759929,-9.303410,-9.951704,0.198734,-4.846475,7.278521
3,accordion,000212_211200.ogg,0.209729,0.338755,0.660264,0.256625,0.275209,0.180417,0.399448,0.381560,...,-0.523156,1.412061,-3.391795,1.884947,-4.441119,6.623763,2.606730,7.237651,0.069198,1.122705
4,accordion,000640_49920.ogg,0.484400,0.334572,0.441629,0.679485,0.382958,0.326033,0.205775,0.309727,...,-8.647794,3.284153,5.265181,10.642460,3.046261,-3.864471,-9.482952,-0.787216,-1.997669,-0.640013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8471,voice,153341_184320.ogg,0.332754,0.359639,0.473210,0.512169,0.607755,0.500391,0.378782,0.420053,...,-4.875653,2.704151,-4.521690,1.879251,-6.319352,1.512789,-7.151809,-1.834733,-7.714043,-2.913627
8472,voice,153412_134400.ogg,0.161793,0.190661,0.249882,0.378161,0.461520,0.332309,0.294171,0.251455,...,-1.182519,-3.178636,-11.015512,-7.479657,-9.194380,-4.153149,-8.789742,4.701506,4.045784,6.724212
8473,voice,153478_30720.ogg,0.589552,0.309548,0.187583,0.169132,0.240637,0.332375,0.333517,0.585136,...,-16.827093,0.815251,-10.296307,-8.400555,-11.928357,-14.542219,-5.466450,-4.281743,-7.056125,6.623515
8474,voice,153986_203520.ogg,0.615495,0.541314,0.467334,0.611199,0.791296,0.678383,0.508162,0.543337,...,0.068194,4.746015,-2.617890,2.208030,-0.682007,2.619602,-1.200114,5.823110,2.130090,8.821922


- Algumas informações

In [27]:
print('Dimensões do dataframe', df.shape)
print('Total de instrumentos na base original:', len(df.instrumento.unique()))
df.instrumento.value_counts()

Dimensões do dataframe (8476, 53)
Total de instrumentos na base original: 20


mallet_percussion    719
flute                654
piano                628
organ                611
saxophone            539
accordion            511
bass                 508
cymbals              503
cello                481
violin               469
voice                452
ukulele              389
banjo                375
synthesizer          375
guitar               347
drums                336
trombone             196
mandolin             163
clarinet             152
trumpet               68
Name: instrumento, dtype: int64

# Preprocessamento

In [28]:
le = LabelEncoder()

df['labels'] = le.fit_transform(df.instrumento)

# Treinando modelos

In [29]:
X = df.drop(columns=['labels', 'file_name'])
y = df['labels']

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [31]:
instrumentos_test = X_test['instrumento']
proporcao_dados_instrumentos_treino = X_train.instrumento.value_counts()

X_train = X_train.drop(columns=['instrumento'])
X_test = X_test.drop(columns=['instrumento'])

- Baseline

In [32]:
models = np.array([KNeighborsClassifier(),DecisionTreeClassifier(), RandomForestClassifier(), LGBMClassifier(),MLPClassifier()])

In [33]:
for model in models:
    model.fit(X_train, y_train)
    predict = model.predict(X_test)
    print(f'Acuracia {model}: {accuracy_score(y_test, predict).round(2) * 100}%')
    

Acuracia KNeighborsClassifier(): 15.0%
Acuracia DecisionTreeClassifier(): 17.0%
Acuracia RandomForestClassifier(): 33.0%
Acuracia LGBMClassifier(): 35.0%
Acuracia MLPClassifier(): 16.0%


# Vamos analisar esses resultados

In [34]:
model = RandomForestClassifier()
model.fit(X_train, y_train)
predict = model.predict(X_test)
print(f'Acuracia Random Forest: {accuracy_score(y_test, predict).round(2) * 100}%')

Acuracia Random Forest: 34.0%


In [35]:
# criar um dataframe de gabarito para auxiliar nas nossas analises
gabarito = X_test
gabarito['labels'] = y_test
gabarito['instrumentos'] = instrumentos_test

# filtrando os acertos
acertos = gabarito[predict == gabarito.labels]

# filtrando os erros
erros = gabarito[predict != gabarito.labels]

- Acerto total

In [36]:
print('Total de acertos: ', acertos.shape[0])

Total de acertos:  858


- Total de instrumentos indentificados

In [37]:
len(acertos.instrumentos.unique())

18

- Instrumentos não indentificados

mandolin e clarinete

- Proporção de dados por instrumentos usado no treino

In [38]:
proporcao_dados_instrumentos_treino

mallet_percussion    491
piano                449
flute                447
organ                437
saxophone            383
bass                 364
cymbals              360
accordion            335
violin               334
cello                316
voice                309
guitar               268
ukulele              261
banjo                258
synthesizer          256
drums                243
trombone             145
mandolin             117
clarinet             108
trumpet               52
Name: instrumento, dtype: int64

- Proporção de instrumentos no Test

In [39]:
gabarito.instrumentos.value_counts()

mallet_percussion    228
flute                207
piano                179
accordion            176
organ                174
cello                165
saxophone            156
bass                 144
voice                143
cymbals              143
violin               135
ukulele              128
synthesizer          119
banjo                117
drums                 93
guitar                79
trombone              51
mandolin              46
clarinet              44
trumpet               16
Name: instrumentos, dtype: int64

- Proporção de acertos por instrumentos

In [40]:
acertos.instrumentos.value_counts()

piano                117
flute                102
cymbals               86
bass                  76
cello                 72
saxophone             66
accordion             47
violin                44
organ                 40
mallet_percussion     39
voice                 38
drums                 36
ukulele               27
banjo                 25
guitar                20
synthesizer           18
trombone               4
trumpet                1
Name: instrumentos, dtype: int64

- Proporção de erros por instrumentos

In [41]:
erros.instrumentos.value_counts()

mallet_percussion    189
organ                134
accordion            129
voice                105
flute                105
synthesizer          101
ukulele              101
cello                 93
banjo                 92
violin                91
saxophone             90
bass                  68
piano                 62
guitar                59
cymbals               57
drums                 57
trombone              47
mandolin              46
clarinet              44
trumpet               15
Name: instrumentos, dtype: int64